# PipelineEndpoint Demo

In [33]:
from azureml.pipeline.wrapper import Module, dsl
from workspace_helpers import setup_default_workspace, load_or_register_dataset
from azureml.pipeline.wrapper._pipeline_endpoint import PipelineEndpoint

In [34]:
workspace = setup_default_workspace()

In [3]:
experiment_name = "TestPipelineEndpoint"
module_func = Module.load(workspace, namespace='azureml', name='Select Columns in Dataset')
test_pipeline_endpoint_name = "test_pipeline_endpoint"
test_pipeline_endpoint_id = "b27eb28c-da34-4064-9c13-e515fdb2c71b"

blob_input_data = load_or_register_dataset(workspace, name='Automobile_price_data_(Raw)',
                                           datastore_name='azureml_globaldatasets',
                                           path='GenericCSV/Automobile_price_data_(Raw)')
select_columns = "{\"isFilter\":true,\"rules\":[{\"exclude\":true,\"ruleType\":\"ColumnNames\",\"columns\":" \
                 "[\"normalized-losses\"]}]}"

@dsl.pipeline(name='test-pipeline', default_compute_target='aml-compute')
def test_pipeline(_select_columns):
    module1 = module_func(
        dataset=blob_input_data,
        select_columns=_select_columns
    )
    return module1.outputs

In [4]:
pipeline = test_pipeline(_select_columns=select_columns)
pipeline_run = pipeline.submit(experiment_name=experiment_name)

Submitted PipelineRun 610950df-3041-4abf-8161-15e95f97d451
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/TestPipelineEndpoint/runs/610950df-3041-4abf-8161-15e95f97d451?wsid=/subscriptions/4faaaf21-663f-4391-96fd-47197c630979/resourcegroups/DesignerTestRG/workspaces/DesignerTest-EUS


### 1. Publish
Create a PipelineEndpoint with the specified name and pipeline/pipeline_run.

#### 1.1 publish by pipeline

In [5]:
pipeline_endpoint_published_by_pipeline = PipelineEndpoint.publish(
    workspace=workspace,
    name=test_pipeline_endpoint_name,
    description="description_for_test_endpoint_publish",
    pipeline=pipeline)

pipeline_endpoint_published_by_pipeline

Using existing pipeline endpoint "test_pipeline_endpoint"
Pipeline endpoint with name "test_pipeline_endpoint" already exist, published pipeline to "test_pipeline_endpoint"


PipelineEndpoint(Name: test_pipeline_endpoint,
Description: None,
Date updated: 2020-08-07 06:54:28.950082+00:00,
Updated by: Liyuan Wu,
Last run submit time: 2020-08-07 06:40:52.606778+00:00,
Last run status: 2,
Status: Active,
rest_endpoint: https://eastus.api.azureml.ms/pipelines/v1.0/subscriptions/4faaaf21-663f-4391-96fd-47197c630979/resourceGroups/DesignerTestRG/providers/Microsoft.MachineLearningServices/workspaces/DesignerTest-EUS/PipelineRuns/PipelineEndpointSubmit/Id/b27eb28c-da34-4064-9c13-e515fdb2c71b)

#### 1.2 publish by pipeline_run

In [6]:
# pipeline_run test
pipeline_endpoint_published_by_pipeline_run = PipelineEndpoint.publish(
    workspace=workspace,
    name=test_pipeline_endpoint_name,
    description="description_for_test_endpoint_publish",
    pipeline=pipeline_run
)

pipeline_endpoint_published_by_pipeline_run

method publish_to_endpoint: This is an experimental method, and may change at any time.<br/>For more information, see https://aka.ms/azuremlexperimental.


Using existing pipeline endpoint "test_pipeline_endpoint"
Pipeline endpoint with name "test_pipeline_endpoint" already exist, published pipeline to "test_pipeline_endpoint"


PipelineEndpoint(Name: test_pipeline_endpoint,
Description: None,
Date updated: 2020-08-07 06:54:46.040358+00:00,
Updated by: Liyuan Wu,
Last run submit time: 2020-08-07 06:40:52.606778+00:00,
Last run status: 2,
Status: Active,
rest_endpoint: https://eastus.api.azureml.ms/pipelines/v1.0/subscriptions/4faaaf21-663f-4391-96fd-47197c630979/resourceGroups/DesignerTestRG/providers/Microsoft.MachineLearningServices/workspaces/DesignerTest-EUS/PipelineRuns/PipelineEndpointSubmit/Id/b27eb28c-da34-4064-9c13-e515fdb2c71b)

### 2. Get pipeline endpoint
Get the PipelineEndpoint by name or ID

#### 2.1 get by id

In [7]:
endpoint_got_by_id = PipelineEndpoint.get(workspace=workspace, id=test_pipeline_endpoint_id)

endpoint_got_by_id

PipelineEndpoint(Name: test_pipeline_endpoint,
Description: None,
Date updated: 2020-08-07 06:54:46.040358+00:00,
Updated by: Liyuan Wu,
Last run submit time: 2020-08-07 06:40:52.606778+00:00,
Last run status: 2,
Status: Active,
rest_endpoint: https://eastus.api.azureml.ms/pipelines/v1.0/subscriptions/4faaaf21-663f-4391-96fd-47197c630979/resourceGroups/DesignerTestRG/providers/Microsoft.MachineLearningServices/workspaces/DesignerTest-EUS/PipelineRuns/PipelineEndpointSubmit/Id/b27eb28c-da34-4064-9c13-e515fdb2c71b)

#### 2.2 get by name

In [8]:
endpoint_got_by_name = PipelineEndpoint.get(workspace=workspace, name=test_pipeline_endpoint_name)

endpoint_got_by_name

PipelineEndpoint(Name: test_pipeline_endpoint,
Description: None,
Date updated: 2020-08-07 06:54:46.040358+00:00,
Updated by: Liyuan Wu,
Last run submit time: 2020-08-07 06:40:52.606778+00:00,
Last run status: 2,
Status: Active,
rest_endpoint: https://eastus.api.azureml.ms/pipelines/v1.0/subscriptions/4faaaf21-663f-4391-96fd-47197c630979/resourceGroups/DesignerTestRG/providers/Microsoft.MachineLearningServices/workspaces/DesignerTest-EUS/PipelineRuns/PipelineEndpointSubmit/Id/b27eb28c-da34-4064-9c13-e515fdb2c71b)

#### 2.3 get not-exist endpoint

In [9]:
PipelineEndpoint.get(workspace=workspace, name="pipeline_endpoint_not_exist")

UserErrorException: UserErrorException:
	Message: Pipeline Endpoint "pipeline_endpoint_not_exist" not exist.
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Pipeline Endpoint \"pipeline_endpoint_not_exist\" not exist."
    }
}

### 3. Pipeline endpoint Submit
Submit default version of endpoint.

In [10]:
test_submit_experiment_name = "test_submit_by_pipeline_endpoint"
pipeline_endpoint = PipelineEndpoint.get(workspace=workspace, id=test_pipeline_endpoint_id)

run_submitted_by_pipeline_endpoint = pipeline_endpoint.submit(experiment_name=test_submit_experiment_name)

Submit pipeline test_pipeline_endpoint use default parameters {'_select_columns': '{"isFilter":true,"rules":[{"exclude":true,"ruleType":"ColumnNames","columns":["normalized-losses"]}]}'}
Submitted PipelineEndpointRun 6c052484-6210-48c9-b6de-b7b9852df7e5
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/test_submit_by_pipeline_endpoint/runs/6c052484-6210-48c9-b6de-b7b9852df7e5?wsid=/subscriptions/4faaaf21-663f-4391-96fd-47197c630979/resourcegroups/DesignerTestRG/workspaces/DesignerTest-EUS


In [11]:
run_submitted_by_pipeline_endpoint.tags["azureml.pipelineid"]

'19d79c94-279d-486f-b7f8-5014bd5e2071'

In [12]:
pipeline_endpoint._default_pipeline_id

'19d79c94-279d-486f-b7f8-5014bd5e2071'

### 4. List
List active PipelineEndpoints in the current workspace.

In [13]:
pipeline_endpoints_list = PipelineEndpoint.list(workspace=workspace, active_only=True)

In [14]:
pipeline_endpoints_list

[PipelineEndpoint(Name: test_pipeline_endpoint,
 Description: None,
 Date updated: 2020-08-07 06:54:46.040358+00:00,
 Updated by: Liyuan Wu,
 Last run submit time: 2020-08-07 06:55:35.987182+00:00,
 Last run status: 0,
 Status: Active,
 rest_endpoint: https://eastus.api.azureml.ms/pipelines/v1.0/subscriptions/4faaaf21-663f-4391-96fd-47197c630979/resourceGroups/DesignerTestRG/providers/Microsoft.MachineLearningServices/workspaces/DesignerTest-EUS/PipelineRuns/PipelineEndpointSubmit/Id/b27eb28c-da34-4064-9c13-e515fdb2c71b),
 PipelineEndpoint(Name: test_published_pipeline,
 Description: None,
 Date updated: 2020-08-07 06:46:41.859904+00:00,
 Updated by: 240bbec1-db0f-4c9c-a103-de1dd781f3f2,
 Last run submit time: 2020-08-07 06:40:33.356167+00:00,
 Last run status: 3,
 Status: Active,
 rest_endpoint: https://eastus.api.azureml.ms/pipelines/v1.0/subscriptions/4faaaf21-663f-4391-96fd-47197c630979/resourceGroups/DesignerTestRG/providers/Microsoft.MachineLearningServices/workspaces/DesignerTes

In [15]:
print("pipeline endpoints name list:") 
[endpoint.name for endpoint in pipeline_endpoints_list]

pipeline endpoints name list:


['test_pipeline_endpoint',
 'test_published_pipeline',
 'test_publish_to_pipeline',
 'test_endpoint_publish_endpoint',
 'test_endpoint_publish_endpoint_new',
 'A huge pipeline composed with nodes 1 in 2 outs',
 'pipeline_export_test',
 'test',
 'test-sub-graph',
 'Pipeline-Created-on-7-6-2020111',
 'Pipeline-Created-on-7-6-2020',
 'Sample 3: test2',
 'Sample 3: Binary Classification with Feature Selection - Income Prediction',
 'Sample 1: Regression - Automobile Price Prediction (Basic)']

### 5. Enable & Disable

In [16]:
pipeline_endpoint.enable()
print(pipeline_endpoint.status)

Active


In [17]:
pipeline_endpoint.disable()
print(pipeline_endpoint.status)

Disabled


In [18]:
pipeline_endpoint.enable()
print(pipeline_endpoint.status)

Active


### 6. Get Published Pipeline by version
Get the pipeline of a specified version or the default; throws an exception if the version is not found.

In [19]:
pipeline_version = '0'
pipeline_got = pipeline_endpoint.get_pipeline(version=pipeline_version)

In [20]:
pipeline_got

PublishedPipeline(Name: test_pipeline_endpoint,
Description: None,
Date updated: 2020-08-03 04:21:50.112827+00:00,
Published by: Brynn Yin,
Last run submit time: None,
Last run status: 0,
Status: Disabled,
rest_endpoint: https://eastus.api.azureml.ms/pipelines/v1.0/subscriptions/4faaaf21-663f-4391-96fd-47197c630979/resourceGroups/DesignerTestRG/providers/Microsoft.MachineLearningServices/workspaces/DesignerTest-EUS/PipelineRuns/PipelineSubmit/6296c510-8095-4bcb-a65e-b813757050aa)

In [21]:
# no vwesion is specified, got the default version
pipeline_default = pipeline_endpoint.get_pipeline()

print('pipeline_default.id:', pipeline_default.id)
print('pipeline_endpoint default version:', pipeline_endpoint._default_pipeline_id)

pipeline_default.id: 19d79c94-279d-486f-b7f8-5014bd5e2071
pipeline_endpoint default version: 19d79c94-279d-486f-b7f8-5014bd5e2071


In [22]:
# not exist version
pipeline_version_not_exist = 'not_exist'
pipeline_endpoint.get_pipeline(version=pipeline_version_not_exist)

ValueError: Pipeline version "not_exist" not found in PipelineEndpoint "test_pipeline_endpoint"

### 7. Get pipelines
Get list of PublishedPipelines in PipelineEndpoint, with format {"version": version, "pipeline": PublihsedPipeline}
active_only: Whether to return only active pipelines.

In [23]:
pipelines_list = pipeline_endpoint.get_pipelines_list(active_only=True)

In [28]:
pipelines_list[0]['version']

'220'

In [29]:
pipelines_list[0]['pipeline']

PublishedPipeline(Name: TestPipelineEndpoint-08-07-2020-14-54-published,
Description: TestPipelineEndpointdescription for TestPipelineEndpoint-08-07-2020-14-54-published,
Date updated: 2020-08-07 06:54:46.013822+00:00,
Published by: Liyuan Wu,
Last run submit time: 2020-08-07 06:56:48.519708+00:00,
Last run status: 1,
Status: Active,
rest_endpoint: None)

In [32]:
pipelines_list[0]['pipeline'].disable()
pipelines_list[1]['pipeline'].disable()